In [1]:
import numpy as np

In [15]:
x1 = np.array([
        [1, 0, 0],
        [0.45, 1.24, -0.02]])

x2 = np.array([
        [0.9, -0.1, 0.3],
        [0.8, -0.6, 0.4],
        [0.2, -1.4, -0.4]])

In [56]:
def lp_dist(p:float):
    # _lp_dist has access to its outer function's scope.
    def _lp_dist(x1:np.ndarray, x2:np.ndarray) -> np.ndarray:
        """Get the pairwise Lp-distances between points.
        
        Args:
            x1: an m by d matrix where each row corresponds to a point to
                be compared with each point in x2.    
            x2: an n by d matrix where each row corresponds to a point to
                be compared with each point in x1. 
            p: p > 0

        Returns:
            a distance matrix of dimensions m by n where the (i,j)-th entry
            corresponds to the distance
            between points x1[i, :] and x2[j, :]
        """
        m = x1.shape[0]
        n = x2.shape[0]
        d = x1.shape[1]

        # Compare each row of x1 to each row of x2.
        # To do this, make views x1_view and x2_view
        # into x1 and x2, respectively.  x1_view and x2_view have the same
        # entries as x1 and x2, but they are repeated in a nice
        # configuration. 
        # Example:
        # x1 = np.array([
        # 	[1, 0, 0],
        # 	[0.45, 1.24, -0.02]])
        # x2 = np.array([
        # 	[0.9, -0.1, 0.3],
        # 	[0.8, -0.6, 0.4],
        # 	[0.2, -1.4, -0.4]])
        # x1_view
        # array([[[ 1.  ,  0.  ,  0.  ],
        #     [ 0.45,  1.24, -0.02]],
        #
        #    [[ 1.  ,  0.  ,  0.  ],
        #     [ 0.45,  1.24, -0.02]],
        #
        #    [[ 1.  ,  0.  ,  0.  ],
        #     [ 0.45,  1.24, -0.02]]])
        # x2_view
        # array([[[ 0.9, -0.1,  0.3],
        #         [ 0.8, -0.6,  0.4],
        #         [ 0.2, -1.4, -0.4]],
        #
        #        [[ 0.9, -0.1,  0.3],
        #         [ 0.8, -0.6,  0.4],
        #         [ 0.2, -1.4, -0.4]]])
        
        x1_view = np.broadcast_to(
            array=x1, 
            shape=(n, m, d)
        )
        
        x2_view = np.broadcast_to(
            array=x2, 
            shape=(m, n, d)
        )

        # Make sure our views have the same dimensions.
        x2_view_2 = np.swapaxes(x2_view, axis1=0, axis2=1)
        # Example:
        # x2_view_2
        # array([[[ 0.9, -0.1,  0.3],
        #         [ 0.9, -0.1,  0.3]],
        #
        #        [[ 0.8, -0.6,  0.4],
        #         [ 0.8, -0.6,  0.4]],
        #
        #        [[ 0.2, -1.4, -0.4],
        #         [ 0.2, -1.4, -0.4]]])
        
        # Now, x1_view and x2_view_2 are broadcast-able.

        return np.power(
            np.sum(
                np.power(
                    np.abs(x1_view - x2_view_2), 
                    p
                ), 
                axis=2
            ).T, 
            1.0/p
        )
    
    return _lp_dist

In [17]:
x1

array([[ 1.  ,  0.  ,  0.  ],
       [ 0.45,  1.24, -0.02]])

In [18]:
x2

array([[ 0.9, -0.1,  0.3],
       [ 0.8, -0.6,  0.4],
       [ 0.2, -1.4, -0.4]])

In [55]:
euclidean_dist = lp_dist(p=2)

euclidean_dist(x1, x2)

array([[0.33166248, 0.74833148, 1.66132477],
       [1.44931018, 1.91950514, 2.67889903]])

In [34]:
def lch_to_lab(lch:np.ndarray) -> np.ndarray:
    """Convert LCH color coordinates to LAB.
    
    Args:
        lch: an n by 3 matrix where each
            row is an LCH color.
    """
    h = np.radians(lch[:, 2])
    c = lch[:, 1]
    a = c * np.cos(h)
    b = c * np.sin(h)

    lab_mat = np.empty(shape=(lch.shape[0], lch.shape[1]))

    lab_mat[:, 0] = lch[:, 0]
    lab_mat[:, 1] = a
    lab_mat[:, 2] = b

    return lab_mat

In [57]:
lab_mat = lch_to_lab(np.array([
    [0.77, 0.1549, 113.29],
    [0.77, 0.0999, 146.72],
    [0.77, 0.2, 146.72]
]))

euclidean_dist(lab_mat, lab_mat)

array([[0.        , 0.09025088, 0.11083638],
       [0.09025088, 0.        , 0.1001    ],
       [0.11083638, 0.1001    , 0.        ]])

In [ ]:
def get_medioid(x:np.ndarray, d) -> np.ndarray:
    """Get the medioid using the distance function d.
    
    Args:
        x: an n by p matrix (numpy.ndarray) where each row corresponds to 
            a point in R^p.
        d: a function that takes two matrices (numpy.ndarray) as its first two
            positional arguments.  The function
            should return a matrix (numpy.ndarray) with entries that correspond
            to pairwise distances between rows of the input
            matrices.

    Returns:
        medioid point as a 1-dimensional array
    """
    medioid_index = np.argmin(np.sum(d(x, x), axis=0, keepdims=False))
    return x[medioid_index, :].ravel()

In [61]:
lab_mat

array([[ 0.77      , -0.06124517,  0.14227804],
       [ 0.77      , -0.0835163 ,  0.05481823],
       [ 0.77      , -0.16719979,  0.10974621]])

In [60]:
get_medioid(x=lab_mat, d=euclidean_dist)

array([ 0.77      , -0.0835163 ,  0.05481823])